In [1]:
import json
import pandas as pd

In [2]:
conv_path = '/data/LPJ/ICML25/GraphCoder/ablation_study/fine_tune_text_LLM_with_nl_graph/2k_rtlcoder_gpt4/conversations.jsonl'
conv = pd.read_json(conv_path, lines=True)

In [3]:
conv.iloc[0]

Instruction    Please act as a professional Verilog designer....
Response       module buffer (input in, output out);\n  assig...
Name: 0, dtype: object

In [4]:
for i in range(len(conv)):
    conv.iloc[i]['Instruction'] = conv.iloc[i]['Instruction'][47:]

In [5]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/ablation_study/fine_tune_text_LLM_with_nl_graph/2k_rtlcoder_gpt4/filter_system_prompt_conversations.jsonl'
# conv.to_json(new_conv_path, orient='records', lines=True)

In [6]:
graph_path = '/data/LPJ/ICML25/GraphCoder/ablation_study/fine_tune_text_LLM_with_nl_graph/2k_rtlcoder_gpt4/nl_graph.jsonl'
graph = pd.read_json(graph_path, lines=True)

In [7]:
template = "Please act as a professional Verilog designer. Given a submodules interconnection graph: \n"

In [8]:
for i in range(len(graph)):
    conv.iloc[i]['Instruction'] = template + graph.iloc[i]['nl_graph'] + '\n' + conv.iloc[i]['Instruction']

In [9]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/ablation_study/fine_tune_text_LLM_with_nl_graph/2k_rtlcoder_gpt4/graph_as_prefix/without_module_head/conversations.jsonl'

In [10]:
conv.to_json(new_conv_path, orient='records', lines=True)

In [11]:
import re

def find_module_string(text):
    # Regex pattern to find the string starting with "module" and ending with ";"
    # It uses non-greedy matching to find the shortest match
    pattern = r"module.*?;"
    
    # Using re.DOTALL to make the dot match all characters including newline
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group()  # Returns the matched string
    else:
        return None  # No match found

# # Example usage
# input_text = """
# some initial text
# module name="example1" \n   attribu\nte="   value";
# some other text
# module name="example2" attribute="another value";
# """

# # Find the first occurrence
# result = find_module_string(input_text)
# print("Found substring:", result)

# # If you need to find all occurrences instead of just the first one, you can use re.findall
# all_matches = re.findall(r"module.*?;", input_text, re.DOTALL)
# print("All found substrings:", all_matches)

In [12]:
conv = pd.read_json(new_conv_path, lines=True)

In [13]:
for i in range(len(conv)):
    module_head = find_module_string(conv.iloc[i]['Response'])
    conv.iloc[i]['Instruction'] = conv.iloc[i]['Instruction'] + '\n' + module_head

In [14]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/ablation_study/fine_tune_text_LLM_with_nl_graph/2k_rtlcoder_gpt4/graph_as_prefix/with_module_head/conversations.jsonl'
conv.to_json(new_conv_path, orient='records', lines=True)

In [18]:
conv.iloc[11]['Instruction']

"Please act as a professional Verilog designer. Given a submodules interconnection graph: \nThe graph consists of several nodes and edges with specific attributes and connectivity:\n\n- Nodes:\n  - Node 0: Content is 'clk', type is 'input port'.\n  - Node 1: Content is 'rst', type is 'input port'.\n  - Node 2: Content is 'data_in', type is 'input port'.\n  - Node 3: Content is 'data_out', type is 'output port'.\n  - Node 4: Content is 'valid_in', type is 'input port'.\n  - Node 5: Content is 'valid_out', type is 'output port'.\n  - Node 6: Content is 'module1', type is 'submodule'.\n  - Node 7: Content is 'module2', type is 'submodule'.\n  - Node 8: Content is 'module3', type is 'submodule'.\n\n- Edge Attributes:\n  - There are no specific attributes provided for the edges in the graph.\n\n- Connectivity:\n  - Node 2 ('data_in') is connected to Node 6 ('module1').\n  - Node 0 ('clk') is connected to Node 7 ('module2').\n  - Node 6 ('module1') is connected to Node 7 ('module2').\n  - No

In [17]:
conv.iloc[11]['Response']

"module top_module (\n    input clk,\n    input rst,\n    input [3:0] in,\n    input load,\n    output reg [15:0] data_out,\n    output reg [3:0] count\n);\n\nwire [15:0] decoder_out;\nreg [15:0] shift_reg;\n\n// 4-to-16 decoder module\ndecoder_4to16 decoder (\n    .in(in),\n    .out(decoder_out)\n);\n\n// 16-bit shift register module\nshift_register_16bit shift_reg_inst (\n    .clk(clk),\n    .rst(rst),\n    .load(load),\n    .data_in(decoder_out),\n    .data_out(shift_reg)\n);\n\n// Counter module\ncounter_16bit counter_inst (\n    .data_in(shift_reg),\n    .count(count)\n);\n\nassign data_out = shift_reg;\n\nendmodule\n\n// 4-to-16 decoder module\nmodule decoder_4to16 (\n    input [3:0] in,\n    output [15:0] out\n);\n\nassign out = 1 << in;\n\nendmodule\n\n// 16-bit shift register module\nmodule shift_register_16bit (\n    input clk,\n    input rst,\n    input load,\n    input [15:0] data_in,\n    output reg [15:0] data_out\n);\n\nalways @(posedge clk or posedge rst) begin\n    if 

In [19]:
len(conv)

2149